In [1]:
#@title Install Dependencies

!pip install transformers >> /dev/null
!pip install pyctcdecode >> /dev/null
!pip install https://github.com/kpu/kenlm/archive/master.zip >> /dev/null

In [2]:
#@title Import Libraries
from transformers import AutoProcessor, AutoModelForCTC
import librosa
import torch
import glob
import os
import json
import pandas as pd
import torchaudio.functional as F

In [3]:
# Download sema audio
!wget https://sema-audio-files.s3.eu-west-1.amazonaws.com/audio/SEMA1/SEMA1-2022-10-19T090905-2.wav

--2022-12-14 16:02:18--  https://sema-audio-files.s3.eu-west-1.amazonaws.com/audio/SEMA1/SEMA1-2022-10-19T090905-2.wav
Resolving sema-audio-files.s3.eu-west-1.amazonaws.com (sema-audio-files.s3.eu-west-1.amazonaws.com)... 52.218.41.248, 52.218.116.42, 52.218.20.227, ...
Connecting to sema-audio-files.s3.eu-west-1.amazonaws.com (sema-audio-files.s3.eu-west-1.amazonaws.com)|52.218.41.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333800 (326K) [audio/wav]
Saving to: ‘SEMA1-2022-10-19T090905-2.wav’

SEMA1-2022-10-19T09 100%[===================>] 325.98K   596KB/s    in 0.5s    

2022-12-14 16:02:19 (596 KB/s) - ‘SEMA1-2022-10-19T090905-2.wav’ saved [333800/333800]



In [4]:
audios = glob.glob("*.wav")
audio_file = audios[0]

In [5]:
audio_file 

'SEMA1-2022-10-19T090905-2.wav'

In [9]:
# Load audio file, and get sampling rate
y, sr = librosa.load(audio_file)

In [10]:
# import transformer Models and Preprocessor
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC, Wav2Vec2Processor

model = Wav2Vec2ForCTC.from_pretrained("Sunbird/sunbird-asr")
processor_with_lm = Wav2Vec2ProcessorWithLM.from_pretrained("Sunbird/sunbird-asr")
processor_no_lm = Wav2Vec2Processor.from_pretrained("Sunbird/sunbird-asr")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
resampled_audio = F.resample(torch.tensor(y), sr, 16000).numpy()

In [12]:
input_values = processor_with_lm(resampled_audio, 
                         return_tensors="pt", 
                         sampling_rate=16000).input_values

with torch.no_grad():
    logits = model(input_values).logits

transcription = processor_with_lm.batch_decode(logits.numpy()).text

In [13]:
transcription

['akaato we ate tabula emisana naye kaakula bateeseteese nalagaana']

In [14]:
import IPython.display as ipd
ipd.Audio(audio_file) # load a local WAV file

In [16]:
inputs = processor_no_lm(y, sampling_rate=16000, return_tensors="pt")

with torch.no_grad():
  logits = model(**inputs).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription2 = processor_no_lm.batch_decode(predicted_ids)

In [18]:
transcription2

['taalekaanka sola atnwonafe sa at taafun meniateensingane si aakaafune ate tae  e e sele  aeeda naasi kaagune']